In [1]:
# Dataset
import os
if not os.path.exists('input.txt'):
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
len(text)  # 1M characters

1115394

In [4]:
text[:1000]  # First 1000 characters

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)  # code book?


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### Encode the string to a list of numbers

In [6]:
# create a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("Transformer"))
print(decode(encode("Transformer")))

[32, 56, 39, 52, 57, 44, 53, 56, 51, 43, 56]
Transformer


In [7]:
# # another way of encoding
# import tiktoken
# # tiktok_enc = tiktoken.get_encoding('gpt2')
# tiktok_enc.n_vocab

# print(tiktok_enc.encode("Transformer"))
# print(tiktok_enc.decode([8291, 16354]))

In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)  # Encode and put the data into a tensor
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [9]:
# split the data into train and val

n = int(0.9*len(data)) # 9:1
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8  # context window size

train_data[:block_size+1]  # ordered --> make prediction on every character

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
import numpy as np

x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f"context = {context.tolist()} -> target = {target}")

# Truncate the data so that it can be divided into blocks of block_size

context = [18] -> target = 47
context = [18, 47] -> target = 56
context = [18, 47, 56] -> target = 57
context = [18, 47, 56, 57] -> target = 58
context = [18, 47, 56, 57, 58] -> target = 1
context = [18, 47, 56, 57, 58, 1] -> target = 15
context = [18, 47, 56, 57, 58, 1, 15] -> target = 47
context = [18, 47, 56, 57, 58, 1, 15, 47] -> target = 58


In [12]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8 # maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # random offset to training set
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb, '\n')
print('targets:')
print(yb.shape)
print(yb)

print('\n ---- \n')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) 

targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

 ---- 

when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [

# Model

In [13]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super(BigramLM, self).__init__()
        self.embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, target):
        logits = self.embedding_table(idx)

        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        target = target.view(-1)  # (B, T) -> (B*T)

        loss = F.cross_entropy(logits, target)  # quality of logits wrt target
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLM(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)

torch.Size([32, 65]) tensor(5.0364, grad_fn=<NllLossBackward0>)


#### **Self-attention**

In [14]:
# example


B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [15]:
#version 1

xbow = torch.zeros((B, T, C))  # bag of words: no context, avg over time
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]  # up to t inclusive
        xbow[b, t] = xprev.mean(dim=0)  # average over time

In [16]:
# version 2

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x  # weighted average over time (B, T, T) @ (B, T, C) -> (B, T, C)
assert torch.allclose(xbow, xbow2) # same result

In [17]:
x[0], xbow[0]

(tensor([[ 0.4829, -1.1658],
         [-0.8079, -1.3601],
         [ 1.7531, -0.3309],
         [-0.1625, -0.4791],
         [ 0.7908, -0.4123],
         [ 0.7684, -0.5464],
         [ 1.0996,  1.5933],
         [ 0.1189,  0.4953]]),
 tensor([[ 0.4829, -1.1658],
         [-0.1625, -1.2630],
         [ 0.4761, -0.9523],
         [ 0.3164, -0.8340],
         [ 0.4113, -0.7497],
         [ 0.4708, -0.7158],
         [ 0.5607, -0.3859],
         [ 0.5054, -0.2758]]))

In [18]:
# use matmul

a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()

print(a, '\n\n')
print(b, '\n\n')
print(a@b)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]]) 


tensor([[7., 1.],
        [1., 2.],
        [3., 6.]]) 


tensor([[7.0000, 1.0000],
        [4.0000, 1.5000],
        [3.6667, 3.0000]])


In [19]:
# version 3

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)  # how much of each token from the past to average+aggregate --> not constant --> data dependent --> learnable
wei = wei.masked_fill(tril == 0, float('-inf'))  # token from the future
wei = F.softmax(wei, dim=1)  # normalize to sum to 1

xbow3 = wei @ x
assert torch.allclose(xbow, xbow3) # same result

# weighted aggregation 

In [22]:
# version 4 - 1 head attention

B, T, C = 4, 8, 32  # batch size, time steps, channels
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size)
query = nn.Linear(C, head_size)
value = nn.Linear(C, head_size)

k = key(x)  # (B, T, H)
q = query(x)  # (B, T, H)
wei = q @ k.transpose(-2, -1) #/ head_size**0.5  # (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))  # token from the future
wei = F.softmax(wei, dim=-1)  # normalize to sum to 1

v = value(x)  # (B, T, H)
out = wei @ v  # (B, T, H)

out.shape

torch.Size([4, 8, 16])

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4093, 0.5907, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6125, 0.2334, 0.1541, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6589, 0.1285, 0.0514, 0.1613, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7224, 0.0038, 0.0871, 0.0881, 0.0985, 0.0000, 0.0000, 0.0000],
        [0.1147, 0.3461, 0.1285, 0.2457, 0.1166, 0.0483, 0.0000, 0.0000],
        [0.0672, 0.0092, 0.0313, 0.0558, 0.0156, 0.8094, 0.0114, 0.0000],
        [0.1016, 0.0322, 0.1447, 0.0872, 0.2101, 0.0081, 0.3341, 0.0820]],
       grad_fn=<SelectBackward0>)

### **Why divide by sqrt(d_model)?**

In [33]:
key_example = torch.randn(B, T, head_size)
query_example = torch.randn(B, T, head_size)

print(f"Var of k and q: {key_example.var()}, {query_example.var()}")

Var of k and q: 1.0496212244033813, 1.0038293600082397
